In [2]:
import pandas as pd
import os

# Lees de data dictionary
data_dict = pd.read_excel(r'C:\Users\sanma\Downloads\PORSCH\nieuwe_data_dictionary.xlsx')

# Selecteer relevante variabelen (categorisch, input, meenemen)
relevante_vars = data_dict[(data_dict['Soort data (0=onbekend, 1=continue, 2=categorisch)'] == 2) & 
                           (data_dict['0=input, 1=output, 2=onbekend'] == 0) & 
                           (data_dict['Meenemen (0=nee, 1= ja, 2=onbekend, 3=computatie)'] == 1)]

print(f"Aantal relevante variabelen: {len(relevante_vars)}")
print(relevante_vars['Variable name'].tolist())

# Controleer variabelen zonder bestandsnaam
vars_zonder_bestand = relevante_vars[relevante_vars['Bestandsnaam'].isna()]
print("\nVariabelen zonder bestandsnaam:")
print(vars_zonder_bestand['Variable name'].tolist())

Aantal relevante variabelen: 249
['aanvanders', 'adenocarcinoma', 'adjudat', 'Adjudication_Committee', 'adjuvant', 'adjuvant_treatment', 'adjuvantstart', 'Age_surgery', 'albu', 'amesen', 'anadetail', 'anasinw', 'anastmaagdarm', 'anastomose', 'anasuitw', 'arthepaberrant', 'arthepcomprop', 'arthepdexacces', 'artmesent', 'ASA.high2', 'asascore', 'asascore.compl', 'aspect', 'aspect.2', 'beeldconcl', 'behperdat', 'betrahepati', 'betramesent', 'betranders', 'betrcolon', 'betrmaag', 'betrtrcoel', 'betrvcava', 'betrvporta', 'bildianastomose', 'biliarydrainage', 'biliodig', 'bilirubine', 'BMI', 'ca199', 'Cal_OK_acute_pancreatitis', 'Cal_OK_chylus2016', 'Cal_OK_fistel2016', 'Cal_OK_gallek', 'Cal_OK_GElek', 'cal_wel_geen_PPH', 'cea', 'chylus2016.castor', 'circumcontact', 'Clavien.Dindo.high', 'Clavien_dindo_I_aantal', 'Clavien_Dindo_II_aantal', 'Clavien_Dindo_IIIa_aantal', 'Clavien_Dindo_IIIb_aantal', 'Clavien_Dindo_IVa_aantal', 'Clavien_Dindo_IVb_aantal', 'coelblok', 'colonsegm', 'colont', 'comb

In [7]:

# Functie om patiënt-ID kolom te identificeren
def get_patient_id_column(df):
    possible_id_columns = ['Record.Id', 'Record Id', 'VoorschriftId', 'PatientID']
    for col in possible_id_columns:
        if col in df.columns:
            return col
    return None

# Map met bronbestanden
bronmap = r"C:\Users\sanma\Downloads\PORSCH\PORSCH pancreas map"

In [14]:
import pandas as pd
import os
import numpy as np

def read_csv_safely(file_path):
    encodings = ['utf-8', 'iso-8859-1', 'windows-1252']
    separators = [',', ';', '\t', '|']
    
    for encoding in encodings:
        for sep in separators:
            try:
                return pd.read_csv(file_path, encoding=encoding, sep=sep, low_memory=False)
            except:
                pass
    
    print(f"Kon het bestand niet lezen: {file_path}")
    return None

def get_patient_id_column(df):
    possible_id_columns = ['Record.Id', 'Record Id', 'VoorschriftId', 'PatientID']
    for col in possible_id_columns:
        if col in df.columns:
            return col
    return None

bronmap = r"C:\Users\sanma\Downloads\PORSCH\PORSCH pancreas map"

# Verzamel alle relevante data
all_data = []

for _, row in relevante_vars.iterrows():
    if pd.notna(row['Bestandsnaam']):
        bestandsnamen = row['Bestandsnaam'].split(', ')
        for bestandsnaam in bestandsnamen:
            bestandspad = os.path.join(bronmap, bestandsnaam)
            if os.path.exists(bestandspad):
                df = read_csv_safely(bestandspad)
                if df is not None:
                    id_column = get_patient_id_column(df)
                    if id_column and row['Variable name'] in df.columns:
                        temp_df = df[[id_column, row['Variable name']]].copy()
                        temp_df.columns = ['PatientID', f"{row['Variable name']}_{bestandsnaam}"]
                        all_data.append(temp_df)

def process_data_in_chunks(data_list, chunk_size=100):
    if not data_list:
        raise ValueError("Geen data gevonden om te combineren.")
    
    result = data_list[0]
    for df in data_list[1:]:
        unique_ids = pd.concat([result['PatientID'], df['PatientID']]).unique()
        chunks = np.array_split(unique_ids, max(1, len(unique_ids) // chunk_size))
        
        new_result = pd.DataFrame()
        for chunk in chunks:
            result_chunk = result[result['PatientID'].isin(chunk)]
            df_chunk = df[df['PatientID'].isin(chunk)]
            merged_chunk = pd.merge(result_chunk, df_chunk, on='PatientID', how='outer')
            new_result = pd.concat([new_result, merged_chunk], ignore_index=True)
        
        result = new_result
        
        # Vrij geheugen op
        del new_result
        import gc
        gc.collect()
    
    return result

# Verwerk data in chunks
input_data = process_data_in_chunks(all_data)

# Verwijder dubbele kolommen
input_data = input_data.loc[:, ~input_data.columns.duplicated()]

# Identificeer dubbele patiënt-ID's
duplicate_counts = input_data['PatientID'].value_counts()
max_duplicates = duplicate_counts.max()

# Maak een Excel schrijver object
excel_writer = pd.ExcelWriter(r'C:\Users\sanma\Downloads\PORSCH\input_data.xlsx', engine='openpyxl')

# Schrijf de hoofddata (eerste voorkomen van elke patiënt)
input_data.drop_duplicates(subset=['PatientID'], keep='first').to_excel(excel_writer, sheet_name='Hoofddata', index=False)

# Schrijf dubbele gegevens naar aparte bladen
for i in range(2, max_duplicates + 1):
    duplicate_ids = duplicate_counts[duplicate_counts >= i].index
    duplicate_data = input_data[input_data['PatientID'].isin(duplicate_ids)].drop_duplicates(subset=['PatientID'], keep='last')
    if not duplicate_data.empty:
        duplicate_data.to_excel(excel_writer, sheet_name=f'Dubbele_Data_{i}', index=False)
    input_data = input_data[~input_data['PatientID'].isin(duplicate_ids)]

# Sla het Excel bestand op
excel_writer.save()
print(f"Data opgeslagen in {excel_writer.path}")

# Analyseer missing data in de hoofddata
main_data = pd.read_excel(r'C:\Users\sanma\Downloads\PORSCH\input_data.xlsx', sheet_name='Hoofddata')
missing_percentages = main_data.isnull().mean() * 100

# Toon kolommen met meer dan 50% missing data
high_missing = missing_percentages[missing_percentages > 50].sort_values(ascending=False)
print("Kolommen met meer dan 50% missing data:")
print(high_missing)

# Verwijder kolommen met meer dan 50% missing data, behalve de ID kolom
columns_to_drop = high_missing.index.drop('PatientID', errors='ignore')
input_data_clean = main_data.drop(columns=columns_to_drop)
input_data_clean.to_excel(r'C:\Users\sanma\Downloads\PORSCH\input_data_clean.xlsx', index=False)
print(f"Opgeschoonde input data opgeslagen met {len(input_data_clean.columns)} kolommen.")

In [16]:
import pandas as pd
import os
import sqlite3

def read_csv_safely(file_path):
    encodings = ['utf-8', 'iso-8859-1', 'windows-1252']
    separators = [',', ';', '\t', '|']
    
    for encoding in encodings:
        for sep in separators:
            try:
                return pd.read_csv(file_path, encoding=encoding, sep=sep, low_memory=False)
            except:
                pass
    
    print(f"Kon het bestand niet lezen: {file_path}")
    return None

def get_patient_id_column(df):
    possible_id_columns = ['Record.Id', 'Record Id', 'VoorschriftId', 'PatientID']
    for col in possible_id_columns:
        if col in df.columns:
            return col
    return None

def column_exists(cursor, table_name, column_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    return any(row[1] == column_name for row in cursor.fetchall())

# Lees de data dictionary
data_dict = pd.read_excel(r'C:\Users\sanma\Downloads\PORSCH\nieuwe_data_dictionary.xlsx')

# Print de kolomnamen om te zien wat we hebben
print("Kolomnamen in de data dictionary:")
print(data_dict.columns)

# Zoek de juiste kolomnamen
soort_data_col = 'Soort data (0=onbekend, 1=continue, 2=categorisch)'
input_output_col = '0=input, 1=output, 2=onbekend'
meenemen_col = 'Meenemen (0=nee, 1= ja, 2=onbekend, 3=computatie)'

# Selecteer relevante variabelen (categorisch, input, meenemen)
relevante_vars = data_dict[(data_dict[soort_data_col] == 2) & 
                           (data_dict[input_output_col] == 0) & 
                           (data_dict[meenemen_col] == 1)]

print(f"Aantal relevante variabelen: {len(relevante_vars)}")

bronmap = r"C:\Users\sanma\Downloads\PORSCH\PORSCH pancreas map"

# Maak een SQLite database
conn = sqlite3.connect(r'C:\Users\sanma\Downloads\PORSCH\porsch_data.db')
cursor = conn.cursor()

# Maak een tabel voor de hoofddata
cursor.execute('''
CREATE TABLE IF NOT EXISTS hoofddata
(PatientID TEXT PRIMARY KEY)
''')

# Verwerk de data
for _, row in relevante_vars.iterrows():
    if pd.notna(row['Bestandsnaam']):
        bestandsnamen = row['Bestandsnaam'].split(', ')
        for bestandsnaam in bestandsnamen:
            bestandspad = os.path.join(bronmap, bestandsnaam)
            if os.path.exists(bestandspad):
                df = read_csv_safely(bestandspad)
                if df is not None:
                    id_column = get_patient_id_column(df)
                    if id_column and row['Variable name'] in df.columns:
                        # Voeg de kolom toe aan de tabel als deze nog niet bestaat
                        column_name = f"{row['Variable name']}_{bestandsnaam}"
                        if not column_exists(cursor, 'hoofddata', column_name):
                            cursor.execute(f"ALTER TABLE hoofddata ADD COLUMN '{column_name}' TEXT")
                        
                        # Update de data in de tabel
                        for _, data_row in df.iterrows():
                            patient_id = data_row[id_column]
                            value = data_row[row['Variable name']]
                            cursor.execute(f'''
                            INSERT OR REPLACE INTO hoofddata (PatientID, '{column_name}')
                            VALUES (?, ?)
                            ''', (patient_id, value))

        conn.commit()

# Sluit de database verbinding
conn.close()

print("Data opgeslagen in SQLite database.")

# Analyseer missing data
conn = sqlite3.connect(r'C:\Users\sanma\Downloads\PORSCH\porsch_data.db')
main_data = pd.read_sql_query("SELECT * FROM hoofddata", conn)
conn.close()

missing_percentages = main_data.isnull().mean() * 100

# Toon kolommen met meer dan ..% missing data
high_missing = missing_percentages[missing_percentages > 99].sort_values(ascending=False)
print("Kolommen met meer dan 80% missing data:")
print(high_missing)

# Verwijder kolommen met meer dan ..% missing data, behalve de ID kolom
columns_to_keep = ['PatientID'] + list(missing_percentages[missing_percentages <= 99].index)
input_data_clean = main_data[columns_to_keep]

# Sla de opgeschoonde data op in een nieuw Excel bestand
input_data_clean.to_excel(r'C:\Users\sanma\Downloads\PORSCH\input_data_clean.xlsx', index=False)
print(f"Opgeschoonde input data opgeslagen met {len(input_data_clean.columns)} kolommen.")

Kolomnamen in de data dictionary:
Index(['Variable name', 'Field label', 'Variable Value',
       'Meenemen (0=nee, 1= ja, 2=onbekend, 3=computatie)',
       '0=input, 1=output, 2=onbekend', 'Opmerkingen', 'Potentiele interactie',
       'Computatie nodig',
       'Soort data (0=onbekend, 1=continue, 2=categorisch)', 'Bestandsnaam'],
      dtype='object')
Aantal relevante variabelen: 249
Data opgeslagen in SQLite database.
Kolommen met meer dan 80% missing data:
aanvanders_data.full.DCPAincluded.csv                       100.0
notPDAC.or.itis_data.full.DCPAincluded.csv                  100.0
origine_data.full.DCPAincluded.csv                          100.0
operative_time_PORSCH_export_20210329.csv                   100.0
operative_time_data.full.DCPAincluded_aangepast.csv         100.0
                                                            ...  
component_data.full.DCPAincluded.csv                        100.0
comorbiditeiten_uri_data.full.DCPAincluded_aangepast.csv    100.0
comor